In [1]:
import pandas as pd
import glob
import json
import requests

# 지하철 역별 위치 정보 저장 (위도, 경도, 행정구역 명)

In [2]:
# `sub_yx` 지하철 위경도 좌표가 적힌 df 생성
sub_yx = pd.read_csv('data/subway_day/지하철역_좌표.csv', encoding='cp949')

In [3]:
# `sub_yx` df에 좌표데이터와 매칭되는 행정구역 명 추가
api_key = '2c56f9597f650d4a2dec845b2e0fe615'

# 위도 경도로 주소를 찾아서 리스트에 넣어주는 함수
def lat_lon_to_addr(lon,lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=lon,latitude=lat)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address_name']
    return addr.append(match_first)

addr = []

for i in range(0,len(sub_yx['X좌표'])):
    try: lat_lon_to_addr(sub_yx['X좌표'][i],sub_yx['Y좌표'][i])
    except :
        addr.append(None)
        
# #리스트를 데이터 프레임으로 저장
addr = pd.DataFrame(addr)

#컬럼 정리
addr['X좌표'] = sub_yx['X좌표']
addr.rename(columns={0:'주소'},inplace=True)
addr=addr[['X좌표', '주소']]

#병합하고 분리
sub_yx = pd.merge(sub_yx,addr, on = 'X좌표', how='left')

sub_yx = sub_yx[sub_yx['주소'].str.split(' ').str.get(0) == '서울특별시']

sub_yx['구'] = sub_yx['주소'].str.split(' ').str.get(-2)
sub_yx['동'] = sub_yx['주소'].str.split(' ').str.get(-1)


# 저장해서 로드 시간을 단축
sub_yx.to_csv('data/subway_day/지하철역_위치데이터.csv', index=False)

### (지하철 위치데이터 `sub_yx` 데이터프레임 정보)

In [4]:
type(sub_yx)

pandas.core.frame.DataFrame

In [5]:
sub_yx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 581
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      297 non-null    object 
 1   Y좌표     297 non-null    float64
 2   X좌표     297 non-null    float64
 3   주소      297 non-null    object 
 4   구       297 non-null    object 
 5   동       297 non-null    object 
dtypes: float64(2), object(4)
memory usage: 16.2+ KB


In [6]:
sub_yx.dtypes

역명      object
Y좌표    float64
X좌표    float64
주소      object
구       object
동       object
dtype: object

In [7]:
sub_yx.size

1782

In [8]:
sub_yx.ndim

2

In [9]:
sub_yx.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            548, 549, 550, 556, 576, 577, 578, 579, 580, 581],
           dtype='int64', length=297)

In [10]:
sub_yx.values

array([['가락시장', 37.492522, 127.118234, '서울특별시 송파구 가락동', '송파구', '가락동'],
       ['종로3가', 37.571607, 126.991806, '서울특별시 종로구 묘동', '종로구', '묘동'],
       ['오금', 37.502162, 127.128111, '서울특별시 송파구 오금동', '송파구', '오금동'],
       ...,
       ['솔샘', 37.62124, 127.013528, '서울특별시 강북구 미아동', '강북구', '미아동'],
       ['북한산보국문', 37.612343, 127.008009, '서울특별시 성북구 정릉동', '성북구', '정릉동'],
       ['정릉', 37.602798, 127.01349, '서울특별시 성북구 정릉동', '성북구', '정릉동']],
      dtype=object)

In [11]:
sub_yx.columns

Index(['역명', 'Y좌표', 'X좌표', '주소', '구', '동'], dtype='object')

In [12]:
sub_yx.head(2)

,역명,Y좌표,X좌표,주소,구,동
0,가락시장,37.492522,127.118234,서울특별시 송파구 가락동,송파구,가락동
1,종로3가,37.571607,126.991806,서울특별시 종로구 묘동,종로구,묘동


In [13]:
sub_yx.tail(2)

,역명,Y좌표,X좌표,주소,구,동
580,북한산보국문,37.612343,127.008009,서울특별시 성북구 정릉동,성북구,정릉동
581,정릉,37.602798,127.013490,서울특별시 성북구 정릉동,성북구,정릉동


In [14]:
sub_yx['역명'].nunique()

295

# 2017년_지하철데이터 생성

##  1. 2017년도 월별 지하철 지하철탑승정보 csv 파일 병합

In [15]:
# csv 파일을 불러 병합
csvfiles = glob.glob('data/subway_day/2017/*.csv')
# `sub_off2017` 2017년도 지하철 하차인원 csv 파일의 합 df 생성
sub_off2017 = pd.DataFrame()

for csvfile in csvfiles:
    tmp = pd.read_csv(csvfile, encoding='cp949', low_memory = False)
    sub_off2017 = pd.concat([sub_off2017, tmp])
    
sub_off2017.reset_index(drop=True, inplace=True)

### (`sub_off2017` 데이터프레임 정보)

In [16]:
type(sub_off2017)

pandas.core.frame.DataFrame

In [17]:
sub_off2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208257 entries, 0 to 208256
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   사용일자    208257 non-null  int64 
 1   노선명     208257 non-null  object
 2   역명      208257 non-null  object
 3   승차총승객수  208257 non-null  int64 
 4   하차총승객수  208257 non-null  int64 
 5   등록일자    208257 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 9.5+ MB


In [18]:
sub_off2017.dtypes

사용일자       int64
노선명       object
역명        object
승차총승객수     int64
하차총승객수     int64
등록일자       int64
dtype: object

In [19]:
sub_off2017.size

1249542

In [20]:
sub_off2017.ndim

2

In [21]:
sub_off2017.index

RangeIndex(start=0, stop=208257, step=1)

In [22]:
sub_off2017.values

array([[20170101, '2호선', '봉천', 11169, 11198, 20170109],
       [20170101, '2호선', '신림', 40137, 42005, 20170109],
       [20170101, '2호선', '신대방', 16033, 16054, 20170109],
       ...,
       [20171231, '1호선', '시청', 17749, 17461, 20180103],
       [20171231, '1호선', '서울역', 41637, 42623, 20180103],
       [20171231, '우이신설선', '신설동', 913, 1031, 20180103]], dtype=object)

In [23]:
sub_off2017.columns

Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')

In [24]:
sub_off2017.head(2)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20170101,2호선,봉천,11169,11198,20170109
1,20170101,2호선,신림,40137,42005,20170109


In [25]:
sub_off2017.tail(2)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
208255,20171231,1호선,서울역,41637,42623,20180103
208256,20171231,우이신설선,신설동,913,1031,20180103


In [26]:
# 총 지하철역 개수 파악
sub_off2017['역명'].nunique()

503

## 2. 데이터 정제 및 저장

In [27]:
# 불필요한 칼럼 제거
sub_off2017.drop(['노선명', '승차총승객수', '등록일자'], axis=1, inplace=True)

In [28]:
# `역명` 칼럼을 기준으로 sub_off2017에 sub_yx에서 역명에 일치하는 위도, 경도값, 행정구역 명 병합
sub_2017 = sub_off2017.merge(sub_yx, on='역명', how='left')

In [29]:
# 주소값이 매칭되지 않아서 서울시 이외의 `역명` 컬럼에 null 값이 들어간 행 제거
sub_2017.dropna(axis=0, inplace = True)

In [30]:
sub_2017.head(2)

,사용일자,역명,하차총승객수,Y좌표,X좌표,주소,구,동
0,20170101,봉천,11198,37.482362,126.941892,서울특별시 관악구 봉천동,관악구,봉천동
1,20170101,신림,42005,37.484201,126.929715,서울특별시 관악구 신림동,관악구,신림동


In [31]:
sub_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130030 entries, 0 to 208986
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    130030 non-null  int64  
 1   역명      130030 non-null  object 
 2   하차총승객수  130030 non-null  int64  
 3   Y좌표     130030 non-null  float64
 4   X좌표     130030 non-null  float64
 5   주소      130030 non-null  object 
 6   구       130030 non-null  object 
 7   동       130030 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 8.9+ MB


In [32]:
sub_2017['역명'].nunique()

289

In [33]:
# `int64` 타입이였던 `사용일자` 컬럼의 데이터들을 `string` 타입으로 변환 한 후에 
# 인덱싱을 통해 `년`, `월`, `일` 형태에 맞게 컬럼으로 추가
sub_2017['년'] = sub_2017['사용일자'].astype('string').str[:4]
sub_2017['월'] = sub_2017['사용일자'].astype('string').str[4:6]
sub_2017['일'] = sub_2017['사용일자'].astype('string').str[6:]

In [34]:
# 컬럼의 순서를 다른 데이터와 합치기 좋게 배치
sub_2017 = sub_2017[['사용일자', '년', '월', '일', '역명', '하차총승객수', 'X좌표', 'Y좌표', '주소', '구', '동']]

In [35]:
# `역명` 컬럼에 혹시나 좌표값이 들어가지 않은 열이 있는지 `isnull`을 통해서 검토
sub_2017['역명'].isnull().sum()

0

In [36]:
# 만약 `역명` 컬럼에 `null`이 있는지 `isnull`로 개수를 알아봤다면
# `null` 값이 들어간 해당 열이 어딘지 확인
# sub_2017[sub_2017['역명'].isnull()]

In [37]:
# 정제가 끝난 후 파일을 마지막으로 df 검토
sub_2017.head()

,사용일자,년,월,일,역명,하차총승객수,X좌표,Y좌표,주소,구,동
0,20170101,2017,01,01,봉천,11198,126.941892,37.482362,서울특별시 관악구 봉천동,관악구,봉천동
1,20170101,2017,01,01,신림,42005,126.929715,37.484201,서울특별시 관악구 신림동,관악구,신림동
2,20170101,2017,01,01,신대방,16054,126.913149,37.487462,서울특별시 관악구 신림동,관악구,신림동
3,20170101,2017,01,01,구로디지털단지,28497,126.901401,37.485266,서울특별시 구로구 구로동,구로구,구로동
4,20170101,2017,01,01,대림(구로구청),21926,126.895801,37.492970,서울특별시 영등포구 대림동,영등포구,대림동


In [39]:
# 정제한 데이터프레임을 csv 파일 형식으로 저장
sub_2017.to_csv('data/subway_total/2017년_지하철데이터.csv',index=False)

# 2018년 지하철데이터 생성

- ( 2018 ~ 2021 년도 ) 데이터 정제 방식은 위의 2017 년도의 과정과 모두 동일

- 2018 년도 `서울역` 승하차 인원 데이터만 원본 데이터에도 기록이 제대로 안된 부분들이 있음

In [40]:
csvfiles = glob.glob('data/subway_day/2018/*.csv')
sub_off2018 = pd.DataFrame()

for csvfile in csvfiles:
    tmp = pd.read_csv(csvfile, encoding='cp949', low_memory = False)
    sub_off2018 = pd.concat([sub_off2018, tmp])
    
sub_off2018.reset_index(drop=True, inplace=True)

In [41]:
sub_off2018.drop(['노선명', '승차총승객수', '등록일자'], axis=1, inplace=True)

In [42]:
sub_2018 = sub_off2018.merge(sub_yx, on='역명')

In [43]:
sub_2018.dropna(axis=0, inplace = True)

In [44]:
sub_2018['년'] = sub_2018['사용일자'].astype('string').str[:4]
sub_2018['월'] = sub_2018['사용일자'].astype('string').str[4:6]
sub_2018['일'] = sub_2018['사용일자'].astype('string').str[6:]

In [45]:
sub_2018 = sub_2018[['사용일자', '년', '월', '일', '역명', '하차총승객수', 'X좌표', 'Y좌표', '주소', '구', '동']]

In [46]:
sub_2018.to_csv('data/subway_total/2018년_지하철데이터.csv',index=False)

# 2019년 지하철데이터 생성

In [47]:
csvfiles = glob.glob('data/subway_day/2019/*.csv')
sub_off2019 = pd.DataFrame()

for csvfile in csvfiles:
    tmp = pd.read_csv(csvfile, encoding='cp949', low_memory = False)
    sub_off2019 = pd.concat([sub_off2019, tmp])
    
sub_off2019.reset_index(drop=True, inplace=True)

In [48]:
sub_off2019.drop(['노선명', '승차총승객수', '등록일자'], axis=1, inplace=True)

In [49]:
sub_2019 = sub_off2019.merge(sub_yx, on='역명')

In [50]:
sub_2019.dropna(axis=0, inplace = True)

In [51]:
sub_2019['년'] = sub_2019['사용일자'].astype('string').str[:4]
sub_2019['월'] = sub_2019['사용일자'].astype('string').str[4:6]
sub_2019['일'] = sub_2019['사용일자'].astype('string').str[6:]

In [52]:
sub_2019 = sub_2019[['사용일자', '년', '월', '일', '역명', '하차총승객수', 'X좌표', 'Y좌표', '주소', '구', '동']]

In [53]:
sub_2019.to_csv('data/subway_total/2019년_지하철데이터.csv',index=False)

# 2020년 지하철데이터 생성

In [54]:
csvfiles = glob.glob('data/subway_day/2020/*.csv')
sub_off2020 = pd.DataFrame()

for csvfile in csvfiles:
    tmp = pd.read_csv(csvfile, encoding='cp949', low_memory = False)
    sub_off2020 = pd.concat([sub_off2020, tmp])
    
sub_off2020.reset_index(drop=True, inplace=True)

In [55]:
sub_off2020.drop(['노선명', '승차총승객수', '등록일자'], axis=1, inplace=True)

In [56]:
sub_2020 = sub_off2020.merge(sub_yx, on='역명')

In [57]:
sub_2020.dropna(axis=0, inplace = True)

In [58]:
sub_2020['년'] = sub_2020['사용일자'].astype('string').str[:4]
sub_2020['월'] = sub_2020['사용일자'].astype('string').str[4:6]
sub_2020['일'] = sub_2020['사용일자'].astype('string').str[6:]

In [59]:
sub_2020 = sub_2020[['사용일자', '년', '월', '일', '역명', '하차총승객수', 'X좌표', 'Y좌표', '주소', '구', '동']]

In [60]:
sub_2020.to_csv('data/subway_total/2020년_지하철데이터.csv',index=False)

# 2021년 지하철데이터 생성

In [61]:
csvfiles = glob.glob('data/subway_day/2021/*.csv')
sub_off2021 = pd.DataFrame()

for csvfile in csvfiles:
    tmp = pd.read_csv(csvfile, encoding='cp949', low_memory = False)
    sub_off2021 = pd.concat([sub_off2021, tmp])
    
sub_off2021.reset_index(drop=True, inplace=True)

In [62]:
sub_off2021.drop(['노선명', '승차총승객수', '등록일자'], axis=1, inplace=True)

In [63]:
sub_2021 = sub_off2021.merge(sub_yx, on='역명')

In [64]:
sub_2021.dropna(axis=0, inplace = True)

In [65]:
sub_2021['년'] = sub_2021['사용일자'].astype('string').str[:4]
sub_2021['월'] = sub_2021['사용일자'].astype('string').str[4:6]
sub_2021['일'] = sub_2021['사용일자'].astype('string').str[6:]

In [66]:
sub_2021 = sub_2021[['사용일자', '년', '월', '일', '역명', '하차총승객수', 'X좌표', 'Y좌표', '주소', '구', '동']]

In [67]:
sub_2021.to_csv('data/subway_total/2021년_지하철데이터.csv',index=False)